## Importing Packages

In [1]:
"""
Importing (using/include) packages and files needed for the code to run

Topo.jl --- Topography Setup

Note: that we import pyplot last to avoid a name conflict with grid
"""

using SparseArrays
using LinearAlgebra
using IterativeSolvers
using WriteVTK
using Printf
using Statistics
using Dates
using HDF5
include("Grid.jl")
include("Markers.jl")
include("Stokes.jl")
include("Temperature.jl")
include("GridOperations.jl")
# Note: that we import pyplot last to avoid a name conflict with grid
using PyPlot
include("Visualization.jl")

plots (generic function with 1 method)

# Topography Setup

In [2]:
function initial_ice_depth(x::Float64,ice_thickness::Float64,wavelength::Float64,amplitude::Float64,initial_surface_depth::Float64)
    return ice_thickness + initial_surface_depth + amplitude*sin( 2*pi/wavelength*x )
end

function ice_viscosity(T::Float64)
    Q = 40000.0 # Activation Enegry (J/mol)
    R_cont = 8.314 # Gas Constant (J/mol*K)
    ice_vis = (1e15)*exp((Q*(273.0-T))/(R_cont*(273.0*T)))
    upperlimit = 1e25
    lowerlimit = 1e12
    if ice_vis < lowerlimit
        ice_vis = lowerlimit
    elseif ice_vis > upperlimit
        ice_vis = upperlimit
    else 
        ice_vis = ice_vis
    end 
    return ice_vis
end

ice_viscosity (generic function with 1 method)

# Material Setup

In [3]:
struct Materials
    # 1 - subsurface global ocean
    # 2 - icy shell
    # 3 - sticky air
    alpha::Vector{Float64} # Thermal expansion (1/K)
    rho0::Vector{Float64} # Density (kg/m^3)
    Hr::Vector{Float64} # Radiogenic heat production (W/m^3)
    Cp::Vector{Float64} # Heat capacity (J/kg*K)
    kThermal::Vector{Float64} # Thermal conductivity (W/m*K)
    eta::Vector{Float64} # Viscosity (Pa*s)
    function Materials()
        new([0.0,0.0,0.0],[1000.0,920.0,1.0],[0.0,0.0,0.0],[4180.0,2100.0,1.0e6],[0.5610,2.1,0.024],[1e12,1e15,1e17])
    end    
end

function update_marker_prop!(markers::Markers,materials::Materials)
    eta = markers.scalarFields["eta"]
    rho = markers.scalarFields["rho"]
    T = markers.scalarFields["T"]
    mmat = markers.integers[markers.integerFields["material"],:]
    for i in 1:markers.nmark
        markers.scalars[rho,i] = materials.rho0[mmat[i]]
        if mmat[i] == 2
            markers.scalars[eta,i] = ice_viscosity(markers.scalars[T,i])
        end
    end
end

function update_marker_temp!(markers::Markers,materials::Materials)
    T = markers.scalarFields["T"]
    mmat = markers.integers[markers.integerFields["material"],:]
    for i in 1:markers.nmark
        if mmat[i] == 1 
            markers.scalars[T,i] = 273.0
        elseif mmat[i] == 3 
            markers.scalars[T,i] = 100.0
        end 
    end 
end 

function initial_conditions!(markers::Markers,materials::Materials,options::Dict)
    material = markers.integerFields["material"]
    T = markers.scalarFields["T"]
    eta = markers.scalarFields["eta"]
    alpha = markers.scalarFields["alpha"]
    Cp = markers.scalarFields["Cp"]
    Hr = markers.scalarFields["Hr"]
    kThermal = markers.scalarFields["kThermal"]
    for i in 1:markers.nmark
        mx = markers.x[1,i]
        my = markers.x[2,i]
        hice = initial_ice_depth(mx,options["ice thickness"],options["wavelength"],options["amplitude"],options["surface depth"])
        hsurf = options["surface depth"]
        if my > hice
            # subsurface global ocean
            markers.integers[material,i] = 1
            markers.scalars[T,i] = 273.0
            markers.scalars[eta,i] = materials.eta[1]
            markers.scalars[alpha,i] = materials.alpha[1]        
            markers.scalars[Cp,i] = materials.Cp[1]
            markers.scalars[Hr,i] = materials.Hr[1] 
            markers.scalars[kThermal,i] = materials.kThermal[1]
        elseif my > hsurf
            # icy shell
            markers.integers[material,i] = 2
            markers.scalars[T,i] = 100.0+((273.0-100.0)/(hice-hsurf))*(my-hsurf)
            # markers.scalars[eta,i] = eta_i[i]
            markers.scalars[alpha,i] = materials.alpha[2]
            markers.scalars[Cp,i] = materials.Cp[2]
            markers.scalars[Hr,i] = materials.Hr[2]
            markers.scalars[kThermal,i] = materials.kThermal[2]
        else
            # sticky air
            markers.integers[material,i] = 3
            markers.scalars[T,i] = 100.0            
            markers.scalars[eta,i] = materials.eta[3]
            markers.scalars[alpha,i] = materials.alpha[3]  
            markers.scalars[Cp,i] = materials.Cp[3]
            markers.scalars[Hr,i] = materials.Hr[3]
            markers.scalars[kThermal,i] = materials.kThermal[3]
        end
    end 
    # end loop over markers
    update_marker_prop!(markers,materials)
end

function get_interface(grid::CartesianGrid,mat::Matrix{Float64},contour_value::Float64)
    # Finding interfaces 
    interface_position = zeros(Float64,grid.nx+1);
    for j in 1:grid.nx+1
        i = 1
        while i <= grid.ny
            if mat[i,j] == contour_value
                interface_position[j] = grid.yc[i]
                break
            elseif mat[i+1,j] < contour_value
                # interface is located within this cell.
                interface_position[j] = grid.yc[i] + (grid.yc[i+1]-grid.yc[i])/(mat[i+1,j]-mat[i,j])*(contour_value-mat[i,j])
                break
            end
            i = i+1
        end
    end
    return interface_position
end

get_interface (generic function with 1 method)

# Model Setup

In [4]:
function run(options::Dict)
    W = options["wavelength"]
    H = options["ice thickness"] + options["surface depth"] + options["amplitude"] + 1e4
    ny = 251
    nx = Int64(ceil(W/H*ny))
        
    gx = 0.0
    gy = 0.113 


    Tbctype = [-1,-1,1,1] #left, right, top, bottom
#     Tbctype = [1,-1,1,1]
    Tbcval = [0.0,0.0,100.0,273.0] #left, right, top, bottom
#     Tbcval = [273.0,0.0,100.0,273.0]
    bc = BoundaryConditions(0,0,0,0) # currently does nothing but is required argument to stokes solver.
    materials = Materials()

    markx = 6
    marky = 6
    seconds_in_year = 3.15e7
    plot_interval = 1e6*seconds_in_year # plot interval in seconds
    end_time = 3e7*seconds_in_year
    dtmax = plot_interval
    grid = CartesianGrid(W,H,nx,ny)
    println("Creating Markers...")
    @time markers = Markers(grid,["alpha","T","rho","eta","Cp","Hr","kThermal"],["material"] ; nmx=markx,nmy=marky,random=true)
    println("Initial condition...")
    @time initial_conditions!(markers, materials,options)

    local time_plot = []
    local max_topo = []
    local topography = []
    ### Setting up agruments for interface function ###
    # initial 
    i_mat, = marker_to_stag(markers,grid,markers.integers[[markers.integerFields["material"]],:],"center")
    i_air_ice_interface = get_interface(grid,i_mat,2.5)
    i_ocean_ice_interface = get_interface(grid,i_mat,1.5)

    ### Setting up agruments for termination criteria ###
    max_step::Int64=100
    max_time::Float64=-1.0
    max_time = max_time == -1.0 ? typemax(Float64) : max_time
    max_step = max_step == -1 ? typemax(Int64) : max_step

    time = 0.0
    iout= 0
    last_plot = 0.0
    dt = 1e10

    rho_c = nothing
    rho_vx = nothing 
    rho_vy = nothing 
    alpha = nothing 
    Hr = nothing 
    Cp_c = nothing 
    eta_s = nothing 
    eta_n = nothing 
    vxc = nothing 
    vyc = nothing 
    T = nothing 
    dTmax = nothing 
    dTemp = nothing 
    Tnew = nothing 
    Tlast = nothing 
    x_time = nothing
    kThermal = nothing
    ocean_ice_interface = nothing
    mat = nothing

    itime = 1
    output_dir = "test"

    terminate = false
    while !terminate
        # 0. update the markers properties  
        update_marker_prop!(markers,materials)
        update_marker_temp!(markers,materials)
        # 1. Transfer properties markers -> nodes
        # 1a. Basic Nodes
        eta_s_new, = marker_to_stag(markers,grid,["eta",],"basic")
        # 1b. Cell Centers
        rho_c_new,Cp_c_new,alpha_new,eta_n_new,Tlast_new,Hr_new,kThermal_new = marker_to_stag(markers,grid,["rho","Cp","alpha","eta","T","Hr","kThermal"],"center")
        # 1c. Vx and Vy nodes:
        rho_vx_new, = marker_to_stag(markers,grid,["rho",],"vx")
        rho_vy_new, = marker_to_stag(markers,grid,["rho",],"vy") 

        # deal with any NaN values from interpolation:
        if itime > 1
            if any(isnan.(eta_s_new))
                println("found nan values")
            end
            replace_nan!(eta_s,eta_s_new)
            replace_nan!(rho_c,rho_c_new)
            replace_nan!(Hr,Hr_new)
            replace_nan!(Cp_c,Cp_c_new)
            replace_nan!(alpha,alpha_new)
            replace_nan!(eta_n,eta_n_new)
            replace_nan!(Tlast,Tlast_new)
            replace_nan!(rho_vx,rho_vx_new)
            replace_nan!(rho_vy,rho_vy_new)
            replace_nan!(kThermal,kThermal_new)
        end
        # Copy field data 
        kThermal = copy(kThermal_new)
        rho_vx = copy(rho_vx_new)
        rho_vy = copy(rho_vy_new)
        rho_c = copy(rho_c_new)
        Hr = copy(Hr_new)
        Cp_c = copy(Cp_c_new)
        alpha = copy(alpha_new)
        eta_s = copy(eta_s_new)
        eta_n = copy(eta_n_new)
        Tlast = copy(Tlast_new)

        if itime == 1 
            println(Tbctype,Tbcval)
            ghost_temperature_center(grid,Tlast,Tbctype,Tbcval)
            cell_center_to_markers!(markers,grid,Tlast,markers.scalars[[markers.scalarFields["T"],],:])
        else
            ghost_temperature_center(grid,Tlast,Tbctype,Tbcval)
        end

        # 2. Assemble and solve the stokes equations
        L,R = form_stokes(grid,eta_s,eta_n,rho_vx,rho_vy,bc,gx,gy;dt=dt)
        stokes_solution = L\R
        vx,vy,P = unpack(stokes_solution,grid;ghost=true)

        # Get the velocity at the cell centers:
        vxc,vyc = velocity_to_centers(grid,vx,vy)
        adiabatic_heating = compute_adiabatic_heating(grid,rho_c,Tlast,alpha,gx,gy,vxc,vyc)*0.0
        shear_heating = compute_shear_heating(grid,vx,vy,eta_n,eta_s)*0.0
        H = (adiabatic_heating .+ shear_heating .+ Hr).*0.0

        # 3. Compute the advection timestep
        if itime > 1
            this_dtmax = min(1.2*dt,dtmax)
        else
            this_dtmax = dtmax
        end
        dt = compute_timestep(grid,vxc,vyc;dtmax=this_dtmax,cfl=0.25)
        diffusion_timestep = (grid.x[2]-grid.x[1])^2 / 1e-6
        if dt > diffusion_timestep
            dt = diffusion_timestep
        end

        dTmax = Inf
        dTemp = nothing
        Tnew = nothing
        titer = 1
        for titer=1:2
            # assemble and solve the energy equation
            println("Trying with timestep ",dt/seconds_in_year/1e3," kyr")
            L,R = assemble_energy_equation_center(grid,rho_c,Cp_c,kThermal,H,Tlast,dt,Tbctype,Tbcval)
            Tnew = L\R;
            Tnew = reshape(Tnew,grid.ny,grid.nx);
            Tnew = ghost_temperature_center(grid,Tnew,Tbctype,Tbcval)
            T = copy(Tnew)

            dTemp = Tnew-Tlast
            # compute the maximum temperature change
            dTmax = maximum(abs.(dTemp[2:end-1,2:end-1]));
            println("dTmax=",dTmax," dt=",dt/seconds_in_year/1e3," kyr")
            dt = min(dt,dTmax < 10.0 ? dt : dt*10.0/dTmax)
            if dTmax < 10.0
                break
            end
        end

        dT_subgrid_node = subgrid_temperature_relaxation_center!(markers,grid,Tlast,Cp_c[1,1],kThermal[1,1],dt)
        dT_remaining = dTemp - dT_subgrid_node

        cell_center_change_to_markers!(markers,grid,dT_remaining,"T")

        # Checking Termination Criteria, time is in Myr
        if time >= max_time || itime >= max_step
            terminate = true
        end    

#         if time == 0.0 || mod(itime,100) == 0 || true
#             last_plot = time 
#             # Gird output
#             name1 = @sprintf("%s/viz.%04d.vtr",output_dir,iout)
#             println("Writing visualization file = ",name1)
#             vn = velocity_to_basic_nodes(grid,vxc,vyc)
#             visualization(grid,rho_c,eta_s,vn,P,Tnew[2:end-1,2:end-1],time/seconds_in_year;filename=name1)
#             # Markers output
#             name2 = @sprintf("%s/markers.%04d.vtp",output_dir,iout)
#             println("Writing visualization file = ",name2)
#             visualization(markers,time/seconds_in_year;filename=name2)
#             iout += 1
#         end

        # println("Min/Max velocity: ",minimum(vyc)," ",maximum(vyc))            
        # Moving the markers and advancing to the next timestep
        move_markers_rk4!(markers,grid,vx,vy,dt,continuity_weight=1/3)
        time += dt
        itime += 1
        println("Finished Step ",itime," time=",time/seconds_in_year/1e3," kyr")
        mat, = marker_to_stag(markers,grid,markers.integers[[markers.integerFields["material"]],:],"center")
        ocean_ice_interface = get_interface(grid,mat,1.5)
        air_ice_interface = get_interface(grid,mat,2.5)
        append!(time_plot,time)
#         append!(max_topo,maximum(ocean_ice_interface.-(options["ice thickness"] + options["surface depth"])))
        append!(topography,[ocean_ice_interface])
    end
    return grid,i_mat,mat,time_plot,topography,max_step
end

run (generic function with 1 method)

In [5]:
include("Topo.jl")
include("FittingData.jl")
function model_run(wavelength::Vector{Float64},ice_shell::Vector{Float64})
    local t_halfspace = []
    local t_viscous = []
    local t_thickening = []
    local lambda = []
    local hice = []
    for i in range(start=1,stop=length(wavelength),step=1)
        options = Dict()
        options["wavelength"] = wavelength[i]
#         options["ice thickness"] = 1e4
        options["ice thickness"] = ice_shell[i]
        options["surface depth"] = 0.50*options["ice thickness"]
        options["amplitude"] = 0.10*options["ice thickness"]

        grid,i_mat,mat,times,topography,max_step = run(options)

        air_ice_interface = get_interface(grid,mat,2.5)
        ocean_ice_interface = get_interface(grid,mat,1.5)
        i_air_ice_interface = get_interface(grid,i_mat,2.5)
        i_ocean_ice_interface = get_interface(grid,i_mat,1.5)

        ths = get_time_viscous(options["wavelength"])
        tvis = get_numerical_time_viscous(i_air_ice_interface,air_ice_interface,i_ocean_ice_interface,ocean_ice_interface,last(times))
        ttic = get_thickening_time(options["ice thickness"])

#         get_topography_plots(grid,i_mat,mat,i_air_ice_interface,air_ice_interface,i_ocean_ice_interface,ocean_ice_interface,topography,times,last(times))
            
#         fitting_data(topography,times,max_step)
        append!(t_halfspace,ths)
        append!(t_viscous,tvis)
        append!(t_thickening,ttic)
        append!(lambda,options["wavelength"])
        append!(hice,options["ice thickness"])
    end
    return lambda,hice,t_viscous,t_thickening,t_halfspace
end

model_run (generic function with 1 method)

In [7]:
wavelength = [1e4,2e4,3e4,4e4]
ice_shell_thickness = [1e4,2e4,3e4,4e4]
run1 = model_run(wavelength,ice_shell_thickness)

Creating Markers...
  0.063665 seconds (19 allocations: 88.991 MiB, 64.98% gc time)
Initial condition...
  0.588890 seconds (7.32 M allocations: 115.163 MiB, 2.57% gc time, 10.37% compilation time)
[-1, -1, 1, 1][0.0, 0.0, 100.0, 273.0]
Trying with timestep 0.34446649029982357 kyr
dTmax=0.9648396052068051 dt=0.34446649029982357 kyr
Finished Step 2 time=0.34446649029982357 kyr
Trying with timestep 0.34446649029982357 kyr
dTmax=0.8471010315801664 dt=0.34446649029982357 kyr
Finished Step 3 time=0.6889329805996471 kyr
Trying with timestep 0.34446649029982357 kyr
dTmax=0.747334739464236 dt=0.34446649029982357 kyr
Finished Step 4 time=1.0333994708994707 kyr
Trying with timestep 0.34446649029982357 kyr
dTmax=0.7676954740344968 dt=0.34446649029982357 kyr
Finished Step 5 time=1.3778659611992943 kyr
Trying with timestep 0.34446649029982357 kyr
dTmax=0.8490686520173654 dt=0.34446649029982357 kyr
Finished Step 6 time=1.7223324514991178 kyr
Trying with timestep 0.34446649029982357 kyr
dTmax=0.93069

Finished Step 58 time=19.634589947089964 kyr
Trying with timestep 0.34446649029982357 kyr
dTmax=1.3688930975881135 dt=0.34446649029982357 kyr
Finished Step 59 time=19.979056437389787 kyr
Trying with timestep 0.34446649029982357 kyr
dTmax=1.3653815798586493 dt=0.34446649029982357 kyr
Finished Step 60 time=20.323522927689613 kyr
Trying with timestep 0.34446649029982357 kyr
dTmax=1.3612318106654016 dt=0.34446649029982357 kyr
Finished Step 61 time=20.667989417989435 kyr
Trying with timestep 0.34446649029982357 kyr
dTmax=1.3565252531419674 dt=0.34446649029982357 kyr
Finished Step 62 time=21.012455908289258 kyr
Trying with timestep 0.34446649029982357 kyr
dTmax=1.3508057623251375 dt=0.34446649029982357 kyr
Finished Step 63 time=21.356922398589084 kyr
Trying with timestep 0.34446649029982357 kyr
dTmax=1.3438678077118311 dt=0.34446649029982357 kyr
Finished Step 64 time=21.701388888888907 kyr
Trying with timestep 0.34446649029982357 kyr
dTmax=1.3357834701947127 dt=0.34446649029982357 kyr
Finish

dTmax=0.3020158651592624 dt=0.31815085597524034 kyr
Finished Step 15 time=4.2364544517679255 kyr
Trying with timestep 0.32106513472042447 kyr
dTmax=0.3239972329547527 dt=0.32106513472042447 kyr
Finished Step 16 time=4.5575195864883495 kyr
Trying with timestep 0.32409554208072716 kyr
dTmax=0.3381438465083306 dt=0.32409554208072716 kyr
Finished Step 17 time=4.881615128569077 kyr
Trying with timestep 0.32721113240438643 kyr
dTmax=0.3462993970301227 dt=0.32721113240438643 kyr
Finished Step 18 time=5.208826260973463 kyr
Trying with timestep 0.33025489830689925 kyr
dTmax=0.3496831210981668 dt=0.33025489830689925 kyr
Finished Step 19 time=5.539081159280363 kyr
Trying with timestep 0.33345159775984223 kyr
dTmax=0.34684646591580304 dt=0.33345159775984223 kyr
Finished Step 20 time=5.872532757040205 kyr
Trying with timestep 0.3367072299654991 kyr
dTmax=0.33911592711001504 dt=0.3367072299654991 kyr
Finished Step 21 time=6.209239987005703 kyr
Trying with timestep 0.34008391368227425 kyr
dTmax=0.325

Trying with timestep 0.8947395701357883 kyr
dTmax=0.7002719696640725 dt=0.8947395701357883 kyr
Finished Step 74 time=33.93594507526977 kyr
Trying with timestep 0.8967172303094908 kyr
dTmax=0.7295854011986194 dt=0.8967172303094908 kyr
Finished Step 75 time=34.83266230557927 kyr
Trying with timestep 0.8967172303094908 kyr
dTmax=0.7342171583737809 dt=0.8967172303094908 kyr
Finished Step 76 time=35.72937953588876 kyr
Trying with timestep 0.8967172303094908 kyr
dTmax=0.7280577644160076 dt=0.8967172303094908 kyr
Finished Step 77 time=36.62609676619825 kyr
Trying with timestep 0.8967172303094908 kyr
dTmax=0.7266538714599733 dt=0.8967172303094908 kyr
Finished Step 78 time=37.52281399650774 kyr
Trying with timestep 0.8967172303094908 kyr
dTmax=0.7428875998818398 dt=0.8967172303094908 kyr
Finished Step 79 time=38.419531226817234 kyr
Trying with timestep 0.8967172303094908 kyr
dTmax=0.7525975666480917 dt=0.8967172303094908 kyr
Finished Step 80 time=39.316248457126726 kyr
Trying with timestep 0.89

Trying with timestep 0.22438423028190624 kyr
dTmax=0.11180455025873925 dt=0.22438423028190624 kyr
Finished Step 31 time=5.951539119335269 kyr
Trying with timestep 0.22660159033368024 kyr
dTmax=0.11408342308311603 dt=0.22660159033368024 kyr
Finished Step 32 time=6.178140709668949 kyr
Trying with timestep 0.22901101738836713 kyr
dTmax=0.11542765650187903 dt=0.22901101738836713 kyr
Finished Step 33 time=6.4071517270573155 kyr
Trying with timestep 0.2315343451996706 kyr
dTmax=0.11578426532497588 dt=0.2315343451996706 kyr
Finished Step 34 time=6.638686072256986 kyr
Trying with timestep 0.2342269997254553 kyr
dTmax=0.11478830105073712 dt=0.2342269997254553 kyr
Finished Step 35 time=6.872913071982442 kyr
Trying with timestep 0.2369883559598786 kyr
dTmax=0.11305320805735164 dt=0.2369883559598786 kyr
Finished Step 36 time=7.10990142794232 kyr
Trying with timestep 0.23958493869814315 kyr
dTmax=0.11061636072651027 dt=0.23958493869814315 kyr
Finished Step 37 time=7.3494863666404635 kyr
Trying with

Trying with timestep 0.7293320572261578 kyr
dTmax=0.33325761183635905 dt=0.7293320572261578 kyr
Finished Step 89 time=28.084994568390762 kyr
Trying with timestep 0.7578698944895328 kyr
dTmax=0.34207183114500594 dt=0.7578698944895328 kyr
Finished Step 90 time=28.842864462880293 kyr
Trying with timestep 0.788870765378251 kyr
dTmax=0.3527007764762402 dt=0.788870765378251 kyr
Finished Step 91 time=29.631735228258545 kyr
Trying with timestep 0.8223812638271766 kyr
dTmax=0.3720431411772438 dt=0.8223812638271766 kyr
Finished Step 92 time=30.45411649208572 kyr
Trying with timestep 0.8587741892074054 kyr
dTmax=0.3912074964696899 dt=0.8587741892074054 kyr
Finished Step 93 time=31.312890681293126 kyr
Trying with timestep 0.8984102677749115 kyr
dTmax=0.40720572267707666 dt=0.8984102677749115 kyr
Finished Step 94 time=32.21130094906804 kyr
Trying with timestep 0.9418052216207697 kyr
dTmax=0.4261361016519345 dt=0.9418052216207697 kyr
Finished Step 95 time=33.15310617068881 kyr
Trying with timestep 0

Finished Step 45 time=6.36167449138758 kyr
Trying with timestep 0.18357950824053554 kyr
dTmax=0.050055758146356766 dt=0.18357950824053554 kyr
Finished Step 46 time=6.545253999628116 kyr
Trying with timestep 0.18587299434610785 kyr
dTmax=0.05023069613127973 dt=0.18587299434610785 kyr
Finished Step 47 time=6.731126993974224 kyr
Trying with timestep 0.18828294843670842 kyr
dTmax=0.05030738244715849 dt=0.18828294843670842 kyr
Finished Step 48 time=6.919409942410932 kyr
Trying with timestep 0.19059643437514448 kyr
dTmax=0.05019404857182508 dt=0.19059643437514448 kyr
Finished Step 49 time=7.110006376786076 kyr
Trying with timestep 0.1930001380980363 kyr
dTmax=0.05002937248059425 dt=0.1930001380980363 kyr
Finished Step 50 time=7.303006514884112 kyr
Trying with timestep 0.19547689485525035 kyr
dTmax=0.049623195125377606 dt=0.19547689485525035 kyr
Finished Step 51 time=7.498483409739364 kyr
Trying with timestep 0.19811131198941068 kyr
dTmax=0.04906911645298351 dt=0.19811131198941068 kyr
Finishe

(Any[10000.0, 20000.0, 30000.0, 40000.0], Any[10000.0, 20000.0, 30000.0, 40000.0], Any[39129.80299751392, 29654.192419005736, 20515.365529887888, 14352.185086749345], Any[2.785910300361167e6, 1.1143641201444669e7, 2.507319270325051e7, 4.4574564805778675e7], Any[4412.969031591225, 2206.4845157956124, 1470.9896771970753, 1103.2422578978062])

In [8]:
wavelength = run1[1][:]
ice_shell = run1[2][:]
timevis = run1[3][:]
timethick = run1[4][:]
timehalfspace = run1[5][:]

4-element Vector{Any}:
 4412.969031591225
 2206.4845157956124
 1470.9896771970753
 1103.2422578978062

In [9]:
dataset = []

Any[]

In [10]:
push!(dataset,[wavelength,ice_shell,timevis,timethick,timehalfspace])

1-element Vector{Any}:
 Vector{Any}[[10000.0, 20000.0, 30000.0, 40000.0], [10000.0, 20000.0, 30000.0, 40000.0], [39129.80299751392, 29654.192419005736, 20515.365529887888, 14352.185086749345], [2.785910300361167e6, 1.1143641201444669e7, 2.507319270325051e7, 4.4574564805778675e7], [4412.969031591225, 2206.4845157956124, 1470.9896771970753, 1103.2422578978062]]

In [11]:
dataset = vcat(map(x->x',dataset)...)

1×5 Matrix{Adjoint{Any, Vector{Any}}}:
 [10000.0 20000.0 30000.0 40000.0]  …  [4412.97 2206.48 1470.99 1103.24]

In [12]:
dataset = hcat(map(x->x',dataset)...)

4×5 Matrix{Float64}:
 10000.0  10000.0  39129.8  2.78591e6  4412.97
 20000.0  20000.0  29654.2  1.11436e7  2206.48
 30000.0  30000.0  20515.4  2.50732e7  1470.99
 40000.0  40000.0  14352.2  4.45746e7  1103.24

In [13]:
using DataFrames
df = DataFrame(lambda = dataset[:,1], hice = dataset[:,2], numerical_time = dataset[:,3], thickening_time = dataset[:,4], halfspace_time = dataset[:,5])

Row,lambda,hice,numerical_time,thickening_time,halfspace_time
,Float64,Float64,Float64,Float64,Float64
1,10000.0,10000.0,39129.8,2.78591e6,4412.97
2,20000.0,20000.0,29654.2,1.11436e7,2206.48
3,30000.0,30000.0,20515.4,2.50732e7,1470.99
4,40000.0,40000.0,14352.2,4.45746e7,1103.24


In [ ]:
figure()
plot(df.lambda./1e4,df.numerical_time./df.halfspace_time,"-o")
gca().set_xlabel(L"\frac{\lambda}{h}")
gca().set_ylabel(L"\frac{t_{model}}{t_{halfspace}}",rotation=0)
gca().yaxis.set_label_coords(-0.2,0.5)
show()

In [16]:
df.hice

4-element Vector{Float64}:
 10000.0
 20000.0
 30000.0
 40000.0

In [17]:
h5open("modelrun.hdf5", "w") do file
    # Creating Groups for Data
    g = create_group(file, "Model Run")
    # Storing Data Inside the Group
    g["Wavelength"] = df.lambda
    g["Relaxation Time(Model)"] = df.numerical_time 
    g["Thickening Time"] = df.thickening_time
    g["Relaxation Time(Halfspace)"] = df.halfspace_time  
    g["Ice Shell Thickness"] = df.hice
    # Apply an Attribute to Groups
    attrs(g)["Description"] = "This group contains only a 4 dataset"
end

"This group contains only a 4 dataset"

In [ ]:
# include("Topo.jl")
# include("FittingData.jl")

# function wavelength_model_run(wavelength::Float64)
    
#     options = Dict()
#     options["surface depth"] = 1e4
#     options["ice thickness"] = 1e4
#     options["wavelength"] = wavelength
#     options["amplitude"] = 1e3
    
#     grid,i_mat,mat,times,topography,max_step = run(options)
    
#     air_ice_interface = get_interface(grid,mat,2.5)
#     ocean_ice_interface = get_interface(grid,mat,1.5)
#     i_air_ice_interface = get_interface(grid,i_mat,2.5)
#     i_ocean_ice_interface = get_interface(grid,i_mat,1.5)

#     ths = get_time_viscous(options["wavelength"])
#     tvis = get_numerical_time_viscous(i_air_ice_interface,air_ice_interface,i_ocean_ice_interface,ocean_ice_interface,last(times))
#     ttic = get_thickening_time(options["ice thickness"])
    
#     get_topography_plots(grid,i_mat,mat,i_air_ice_interface,air_ice_interface,i_ocean_ice_interface,ocean_ice_interface,topography,times,last(times))
#     fitting_data(topography,times,max_step)
    
#     return ths,tvis,ttic
# end

In [ ]:
# wavelength = 10500.0
# test_run = wavelength_model_run(wavelength)

In [ ]:
# push!(half_space,test_run[1])

In [ ]:
# push!(thickening_time,test_run[3])

In [ ]:
# display(num_exp)
# display(half_space)
# display(thickening_time)

In [ ]:
# figure()
# plot(df.lambda./1e4,df.numerical_time./df.halfspace_time,"-o")
# gca().set_xlabel(L"\frac{\lambda}{h}")
# gca().set_ylabel(L"\frac{t_{model}}{t_{halfspace}}",rotation=0)
# gca().yaxis.set_label_coords(-0.1,0.5)
# show()